# Cleanup

After building completing the notebooks you may want to delete the following to prevent any unwanted charges:

* Forecasts
* Predictors
* Datasets
* Dataset Groups

The code snippets below will cover the base use case of creating items in notebooks 1 - 3. You can expand upon this to delete content created in other notebooks.

## Imports and Connections to AWS

The following lines import all the necessary libraries and then connect you to Amazon Forecast.


In [1]:
import sys
import os
import json
import time

import boto3
import pandas as pd

# importing forecast notebook utility from notebooks/common directory
#sys.path.insert( 0, os.path.abspath("../../common") )
import util

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

The line below will retrieve your shared variables from the earlier notebooks.

In [3]:
%store -r

In [4]:
%store -r forecast_dataset_group_arn
%store -r forecast_dataset_arn
%store -r forecast_role_name
%store -r forecast_key
#%store bucket_name
#%store region
%store -r forecast_ds_import_job_arn

%store -r forecast_arn
%store -r forecast_predictor_arn

no stored variable or alias forecast_key
no stored variable or alias forecast_ds_import_job_arn


Once again connect to the Forecast APIs via the SDK.

In [9]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Defining the Things to Cleanup

In the previous notebooks you stored several variables at the end of each, now that they have been retrieved above, the cells below will delete the items that were created one at a time until all items that were created have been removed.

In [6]:
# Delete the Foreacst:
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=forecast_arn))

Successful delete


In [7]:
# Delete the Predictor:
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=forecast_predictor_arn))

Successful delete


In [10]:
# Delete Import
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=forecast_ds_import_job_arn))

NameError: name 'forecast_ds_import_job_arn' is not defined

In [11]:
# Delete the Dataset:
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=forecast_dataset_arn))

ResourceInUseException: An error occurred (ResourceInUseException) when calling the DeleteDataset operation: Operation not allowed when the resource arn:aws:forecast:us-east-1:992382405090:dataset/util_power_forecast_ds is being referenced by these resources : [ arn:aws:forecast:us-east-1:992382405090:dataset-import-job/util_power_forecast_ds/EP_DSIMPORT_JOB_TARGET ]

In [12]:
# Delete the DatasetGroup:
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=forecast_dataset_group_arn))

Successful delete


In [14]:
# # Delete your file in S3
boto3.Session().resource('s3').Bucket(bucket).Object(key).delete()

NameError: name 'key' is not defined

## IAM Role and Policy Cleanup

The very last step in the notebooks is to remove the policies that were attached to a role and then to delete it. No changes should need to be made here, just execute the cell.

In [15]:
util.delete_iam_role(role_name)

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: Policy arn:aws:iam::aws:policy/AmazonForecastFullAccess was not found.

All that remains to cleanup here is to now go back to the CloudFormation console and delete the stack. You have successfully removed all resources that were created.